# 今天我們要來講解下方這個爬取月報的 Function 囉！

# 先來讀資料！

In [1]:
import requests

# 指定爬取月報的網址
url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_106_1_0.html'
# 抓取網頁
r = requests.get(url)

In [2]:
r.text


'<html><script src=\'/nocopy.js\'></script><head><style text=\'text/css\'><!--\t.tt\t{color:#000099;font-weight:normal;font-size:10pt} --> </style><meta http-equiv=\'Content-Type\' content=\'text/html;charset=big5\'></head><body background=\'/image/bg2.gif\'><center><center><font size=\'5\'><b>¤W¥«¤½¥q106¦~1¤ë¥÷(²Ö\xadp»P·í¤ë)Àç·~¦¬¤J²Î\xadpªí</b></font><h3>¥»¸ê®Æ¥Ñ¦U¤½¥q´£¨Ñ</h3><font size=\'1\'><br><br><input type=\'button\' name=\'download\' value=\'¥t¦sCSV\' onClick=\'document.show_file.fileName.value="t21sc03_106_1.csv";document.show_file.submit();\'>(ÀÉ®×¤º®e¥]§t°ê¤º¤Î°ê¥~¤½¥q)</font><form name=\'show_file\' action=\'/server-java/FileDownLoad\' method=\'post\'><input type=\'hidden\' name=\'step\' value=\'9\'><input type=\'hidden\' name=\'functionName\' value=\'show_file2\'><input type=\'hidden\' name=\'filePath\' value=\'/t21/sii/\'><input type=\'hidden\' name=\'fileName\' value=\'\'></form><br><div class=\'tt\' style=\'width:100%;text-align:left;\'>¤W¥«Âd¤½¥q¡B¿³Âd¤½¥q¡B¤Îª÷ºÞ·|

In [3]:
from io import StringIO
import pandas as pd

# 讓pandas可以讀取中文（測試看看，假如不行讀取中文，就改成 'utf-8'）
r.encoding = 'big5'
# 把剛剛下載下來的網頁的 html 文字檔，利用 StringIO() 包裝成一個檔案給 pandas 讀取
dfs = pd.read_html(StringIO(r.text))

In [4]:
# 將dfs中，row長度介於5~11的table合併（這些才是我們需要的table，其他table不需要）
df = pd.concat([df for df in dfs[2:] if df.shape[1] <= 11 and df.shape[1] > 5])

# 設定column名稱 
df.columns = df.columns.get_level_values(1)
df = df.rename(columns={'公司 代號': '公司代號'})

# 將 df 中的當月營收用 .to_numeric 變成數字，再把其中不能變成數字的部分以 NaN 取代（errors='coerce'）
df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')

# 再把當月營收中，出現 NaN 的 row 用 .dropna 整行刪除
df = df[~df['當月營收'].isnull()]

# 刪除「公司代號」中出現「合計」的行數，其中「～」是否定的意思
df = df[df['公司代號'] != '合計']

# # 將「公司代號」與「公司名稱」共同列為 df 的 indexes
df = df.set_index(['公司代號', '公司名稱'])

df.head()

,,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%),備註
公司代號,公司名稱,,,,,,,,,
1101,台泥,6131670,9418581,6575590,-34.89,-6.75,6131670,6575590,-6.75,-
1102,亞泥,4162130,6023954,5066062,-30.90,-17.84,4162130,5066062,-17.84,-
1103,嘉泥,232657,382981,197328,-39.25,17.90,232657,197328,17.90,-
1104,環泥,363865,463842,373311,-21.55,-2.53,363865,373311,-2.53,-
1108,幸福,316773,345573,348266,-8.33,-9.04,316773,348266,-9.04,-


# 存檔csv (全版本通用)

In [5]:
# ----------- #
# 存取 csv 檔  #
# ----------- #

# 把 df 存成 csv 檔，並且命名為「test.csv」，指定用「utf_8_sig」編碼
df.to_csv('test.csv', encoding='utf_8_sig')

# 讀取名為「test.csv」的 csv 檔，並且指定其中欄位名稱為「公司代號」與「公司名稱」作為 df 的 indexes
df = pd.read_csv('test.csv', index_col=['公司代號','公司名稱'])
df.head()

,,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%),備註
公司代號,公司名稱,,,,,,,,,
1101,台泥,6131670,9418581,6575590,-34.89,-6.75,6131670,6575590,-6.75,-
1102,亞泥,4162130,6023954,5066062,-30.90,-17.84,4162130,5066062,-17.84,-
1103,嘉泥,232657,382981,197328,-39.25,17.90,232657,197328,17.90,-
1104,環泥,363865,463842,373311,-21.55,-2.53,363865,373311,-2.53,-
1108,幸福,316773,345573,348266,-8.33,-9.04,316773,348266,-9.04,-


# 存檔sqlite3 (全版本通用)

In [6]:
# --------------- #
# 存取 sqlite3 檔  #
# --------------- #

import sqlite3

# 把 df 存成名為「monthly_report」的 sqlite3 檔，其中 conn 是與 database 的連結
conn = sqlite3.connect('test.sqlite3')
df.to_sql('monthly_report', conn, if_exists='replace')

# 讀取 sqlite3 中名為「monthly_report」的 table，並且指定其中欄位名稱為「公司代號」與「公司代號」作為 df 的 indexes
df = pd.read_sql('select * from monthly_report', conn, index_col=['公司代號','公司名稱'])
df.head()




,,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%),備註
公司代號,公司名稱,,,,,,,,,
1101,台泥,6131670,9418581,6575590,-34.89,-6.75,6131670,6575590,-6.75,-
1102,亞泥,4162130,6023954,5066062,-30.90,-17.84,4162130,5066062,-17.84,-
1103,嘉泥,232657,382981,197328,-39.25,17.90,232657,197328,17.90,-
1104,環泥,363865,463842,373311,-21.55,-2.53,363865,373311,-2.53,-
1108,幸福,316773,345573,348266,-8.33,-9.04,316773,348266,-9.04,-
